# Классификация тональности текста (Text Sentiment Analysis)

**Для определения тональности применим величины TF-IDF как признаки.**

***Анализ тональности текста, или сентимент-анализ (от англ. sentiment, «настроение»), выявляет эмоционально окрашенные слова. Этот инструмент помогает компаниям оценивать, например, реакцию на запуск нового продукта в интернете. На разбор тысячи отзывов человек потратит несколько часов, а компьютер — пару минут.***

Оценить тональность — значит отметить текст как позитивный или негативный. То есть мы решаем задачу классификации, где целевой признак равен «1» для положительного текста и «0» для отрицательного. Признаки — это слова из корпуса и их величины TF-IDF для каждого текста.

### Задача

- Обучите логистическую регрессию так, чтобы она определяла тональность текста.

- Подсчитайте величину TF-IDF для текстов. Лемматизированные тексты твитов для обучения находятся в файле tweets_lemm_train.csv. Целевой признак вы найдёте в столбце positive.

- Обученной моделью классификации определите результаты предсказания для тестовой выборки твитов, которая лежит в файле tweets_lemm_test.csv. В этой выборке целевого признака нет. Сохраните предсказания в столбце positive. Таблицу с результатом сохраните как csv-файл, но чтобы тренажёр принял файл, не указывайте расширение (например, назовите файл 'predictions')

- Значение accuracy вашей модели должно быть не меньше 0.62.
- Для предсказания ответов повторно вычислите величину TF-IDF для вектора с тестовой выборкой. Примените метод transform() к объекту TfidfVectorizer.

In [84]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pymystem3 import Mystem

import nltk
from nltk.corpus import stopwords as nltk_stopwords

import pandas as pd
import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None

### 1 Загрузка данных

In [112]:
train_data = pd.read_csv('/Users/yuliabezginova/PycharmProjects/00_MLForTexts/tweets_lemm_train.csv')
train_data

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...
...,...,...,...
4995,Вроде дружили 10 лет и в один момент все разру...,0,вроде дружить год и в один момент весь разруши...
4996,"@m_gulko \r\nПоэтому и поздравляю заранее, что...",0,поэтому и поздравлять заранее что не получитьс...
4997,"RT @kubudojede: черт, ну зачем они переделали ...",0,черта ну зачем они переделать
4998,"RT @xidewojopiba: Вроде бы и любим друг друга,...",0,вроде бы и любим друг друг и быть вместе не мо...


In [115]:
y_train = train_data['positive']

In [113]:
test_data = pd.read_csv('/Users/yuliabezginova/PycharmProjects/00_MLForTexts/tweets_lemm_test.csv')
test_data

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...
...,...,...
2995,"жуйк, ачивки в контре — зло! мой младший брат ...",жуйк ачивка в контра зло мой младший брат втян...
2996,Не хочу идти на танцы :( http://t.co/5OdPjbYXOC,не хотеть идти на танец
2997,"На улице пусто!ни людей,ни машин,наверно холод...",на улица пусто ни человек ни машина наверно хо...
2998,По-моему мы и ходили за водкой в три часа за э...,по мой мы и ходить за водка в три час за это ч...


### 2 Масштабирование данных с помощью tf-idf

Следующий подход вместо исключения несущественных признаков пытается масштабировать признаки в зависимости от степени их информативности. Одним из наиболее распространенных способов такого масштабирования является метод частота термина-обратная частота документа (term frequency-inverse document frequency, tf-idf). 

Идея этого метода заключается в том, чтобы присвоить большой вес термину, который часто встречается в конкретном документе, но при этом редко встречается в остальных документах корпуса. Если слово часто появляется в конкретном документе, но при этом редко встречается в остальных документах, оно, вероятно, будет описывать содержимое этого документа лучше.

#### В библиотеке scikit-learn метод tf-idf реализован в двух классах: TfidfTransformer, который принимает на вход разреженную матрицу, полученную с помощью CountVectorizer, и преобразует ее, и TfidfVectorizer, который принимает на вход текстовые данные и выполняет как выделение признаков «мешок слов», так и преобразование tf-idf.

Поскольку tf-idf фактически использует статистические свойства
обучающих данных, мы воспользуемся конвейером (о котором
рассказывалось в главе 6), чтобы убедиться в достоверности результатов
нашего решетчатого поиска. Для этого пишем следующий программный
код:

In [146]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuliabezginova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [103]:
# создадим счётчик, указав в нём стоп-слова
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

### 3 Деление данных на подвыборки

In [104]:
%%time
# Чтобы посчитать TF-IDF для корпуса текстов, вызовем функцию fit_transform():
tf_idf_train = count_tf_idf.fit_transform(train_data['lemm_text'])
tf_idf_test = count_tf_idf.transform(test_data['lemm_text'])

CPU times: user 132 ms, sys: 5.82 ms, total: 138 ms
Wall time: 141 ms


In [108]:
print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (5000, 9737)
Размер матрицы: (3000, 9737)


### 4 Обучение логистической регрессии (подбор параметров решетчатым поиском)

In [145]:
logreg = LogisticRegression(random_state=5)

# param_grid = [{"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}]
param_grid = [
    {'C': [0.01, 0.1, 1, 2, 10, 100], 'penalty': ['l1', 'l2']}
]
# =109.85411419875572, class_weight=None, dual=False,
#           fit_intercept=True, intercept_scaling=1, max_iter=100,
#           multi_class='warn', n_jobs=None, penalty='l1', random_state=None,
#           solver='warn', tol=0.0001, verbose=0, warm_start=False

logreg_grid = GridSearchCV(logreg, param_grid, cv=5, verbose=True, n_jobs=-1)

logreg_grid.fit(tf_idf_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/Cellar/jupyterlab/3.4.8/libexec/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=5), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 2, 10, 100],
                          'penalty': ['l1', 'l2']}],
             verbose=True)

### 5 Предсказания на тестовой выборке

In [138]:
predictions = logreg_grid.predict(tf_idf_test)
print("Наилучшее значение перекр проверки: {:.2f}".format(logreg_grid.best_score_))

Наилучшее значение перекр проверки: 0.64


### 6 Запишем файл с предсказаниями на тестовой выборке для загрузки

In [139]:
submission = pd.DataFrame({'positive':predictions})
submission.to_csv('predictions', index=False)

In [140]:
y_pred = pd.read_csv("/Users/yuliabezginova/PycharmProjects/00_MLForTexts/predictions")

***Thank you for going through this project. Your comments are more then welcome to ybezginova2021@gmail.com***

***Best wishes,***

***Yulia***